In [1]:
import geemap
import ee
import os , glob
import datetime
import geopandas as gpd

# Spectral Linear Unmixing

In [2]:
Map = geemap.Map(center = [40.409264,32.770967],zoom=12)
Map.add_basemap('Esri Satellite')
Map


Map(center=[40.409264, 32.770967], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleB…

In [3]:
aoi = ee.Geometry.Point([32.616416, 40.398831])
aoi.getInfo()

{'type': 'Point', 'coordinates': [32.616416, 40.398831]}

## Add points to map

In [4]:
ramsar_shp = '/home/cak/Desktop/lake_extraction/Data/kizilcahamam.shp'
ramsar = geemap.shp_to_ee(ramsar_shp)
Map.addLayer(ramsar, {}, 'Ponds Points')

### Area Definition

In [5]:
area = ee.Geometry.Polygon([[[32.7, 40.300000000000004],
      [33, 40.300000000000004],
      [33, 40.45],
      [32.7, 40.45],
      [32.7, 40.300000000000004]]])
area.getInfo()

{'type': 'Polygon',
 'coordinates': [[[32.7, 40.300000000000004],
   [33, 40.300000000000004],
   [33, 40.45],
   [32.7, 40.45],
   [32.7, 40.300000000000004]]]}

In [6]:
# area_shp = '../Data/Aux/area.shp'
# area = geemap.shp_to_ee(area_shp)
Map.addLayer(area, {}, 'Study Area')
tiles_shp = '../Data/Aux/tiles.shp'
tiles = geemap.shp_to_ee(tiles_shp)
Map.addLayer(tiles, {}, 'Tiles')

In [7]:
# for i in df.iterrows():
#     print(i[1]['geometry'].)
area.getInfo()

{'type': 'Polygon',
 'coordinates': [[[32.7, 40.300000000000004],
   [33, 40.300000000000004],
   [33, 40.45],
   [32.7, 40.45],
   [32.7, 40.300000000000004]]]}

### Landsat Image

In [8]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")


bands = ['B2','B3','B4','B5','B6','B7']



image = landsat \
            .filterBounds(aoi) \
            .sort('CLOUD_COVER') \
            .filterDate('2020-05-01', '2020-05-30')\
            .first() \
            .select(bands)\
            .clip(area)
#             


landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.6,
    'min' : 0,
    'max' : 0.3
}

Map.addLayer(image, landsat_vis, "Landsat_RGB", True, 1)

### Sentinel Image

In [9]:
bands = ['B2','B3','B4','B5','B6','B7','B8','B11','B12']


def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

sentinel = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(aoi) \
                .filterDate('2020-05-01', '2020-05-30') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                .map(maskS2clouds)\
                .select(bands)\
                .first()\
                .clip(area)

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(sentinel, visualization, 'Sentinel_RGB')

In [11]:
bands = ['B2','B3','B4','B5','B6','B7','B8','B11','B12']


def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

temp = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filterBounds(aoi) \
                .filterDate('2020-05-01', '2020-05-30') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                .map(maskS2clouds)\
                .select(bands)
#                 .clip(area)


listOfImages = temp.toList(temp.size());

In [12]:
firstImage = listOfImages.get(0)

In [13]:
firstImage = ee.Image(listOfImages.get(0));
Map.addLayer(firstImage, visualization, 'Sentinel_RGB')

### Global Surface Water

In [14]:
gsw_dataset = ee.Image('JRC/GSW1_2/GlobalSurfaceWater')


gsw_dataset = gsw_dataset.clip(area)

visualization = {
  'bands': ['occurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'ffbbbb', '0000ff']
};

Map.addLayer(gsw_dataset, visualization, 'Surface - Water');

### NDVI and NDWI 

In [15]:
visParams_ndvi = {'min': -0.2, 'max': 0.8, 'palette': 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000,529400,3E8601,207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'};

ndvi = sentinel.expression('(nir - red) / (nir + red)' ,{
    'nir' : sentinel.select('B8'),
    'red' : sentinel.select('B4')
}).rename('NDVI')

ndwi = sentinel.expression('(nir - swir) / (nir + swir)' ,{
    'nir' : sentinel.select('B8'),
    'swir' : sentinel.select('B11')
}).rename('NDWI')

Map.addLayer(ndwi, visParams_ndvi, 'Sentinel_NDWI_man')
Map.addLayer(ndvi, visParams_ndvi, 'Sentinel_NDVI_man')
sentinel = sentinel.addBands(ndvi)
sentinel = sentinel.addBands(ndwi)

## Image Stats

In [16]:
sentinel.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [2551, 1670],
   'origin': [7454, 2225],
   'crs': 'EPSG:32636',
   'crs_transform': [10, 0, 399960, 0, -10, 4500000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [2551, 1670],
   'origin': [7454, 2225],
   'crs': 'EPSG:32636',
   'crs_transform': [10, 0, 399960, 0, -10, 4500000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [2551, 1670],
   'origin': [7454, 2225],
   'crs': 'EPSG:32636',
   'crs_transform': [10, 0, 399960, 0, -10, 4500000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'dimensions': [1276, 836],
   'origin': [3727, 1112],
   'crs':

In [17]:
landsat_props = geemap.image_props(image)
sentinel_props = geemap.image_props(sentinel)
# landsat_props.getInfo()

In [18]:
Date = landsat_props.get('IMAGE_DATE').getInfo()
CC= landsat_props.get('CLOUD_COVER').getInfo()
Zone = landsat_props.get('UTM_ZONE').getInfo()
Scale = landsat_props.get('NOMINAL_SCALE').getInfo()
date_sentinel = sentinel.get('system:index').getInfo()
date_sentinel = datetime.datetime.strptime(date_sentinel.split('_')[0], "%Y%m%dT%H%M%S").date()

In [19]:
from pprint import pprint

Stats = {
    'Date' : Date,
    'Cloud Cover' : "{:.4} %".format(float(CC)*100),
    'UTM Zone ' : Zone, 
    'Resolution' : Scale
}

pprint(Stats)

{'Cloud Cover': '10.0 %',
 'Date': '2020-05-16',
 'Resolution': 30,
 'UTM Zone ': 36}


In [20]:
date_sentinel

datetime.date(2020, 5, 16)

### Create Area of Interest

In [ ]:
# export_area = Map.draw_last_feature

### Export Image

In [ ]:
out_dir = '/home/cak/Desktop/lake_extraction/Data/exported'
filename = os.path.join(out_dir, f'{Date}_landsat_kizil.tif')
filename_water = os.path.join(out_dir, 'surface_water_kizil.tif')
filename_ndvi = os.path.join(out_dir, 'ndvi_kizil.tif')
filename_ndwi = os.path.join(out_dir, 'ndwi_kizil.tif')
filename_gsw = os.path.join(out_dir, 'water_kizil.tif')

In [ ]:
image = image.clip(area).unmask()
ndvi = ndvi.clip(area).unmask()
ndwi = ndwi.clip(area).unmask()
gsw_dataset = gsw_dataset.clip(area).unmask()
geemap.ee_export_image(image, filename=filename, scale=20, file_per_band=False)
geemap.ee_export_image(ndvi, filename=filename_ndvi, scale=10, file_per_band=False)
geemap.ee_export_image(ndwi, filename=filename_ndwi, scale=10, file_per_band=False)
geemap.ee_export_image(gsw_dataset, filename=filename_gsw, scale=20, file_per_band=False)


### Sentinel Export

In [ ]:
sentinel_area = Map.draw_last_feature
sentinel_area.getInfo()

# coordinates': [[[32.800379, 40.374524],
#     [32.800379, 40.442232],
#     [32.942542, 40.442232],
#     [32.942542, 40.374524],
#     [32.800379, 40.374524]]]

In [ ]:
sentinel_area = ee.Geometry.Polygon([[[32.800379, 40.374524],
    [32.800379, 40.442232],
    [32.942542, 40.442232],
    [32.942542, 40.374524],
    [32.800379, 40.374524]]])
area.getInfo()

In [ ]:
tiles = gpd.read_file('../Data/Aux/tiles.shp')

In [ ]:
def coord_lister(geom):
    coords = list(geom.exterior.coords)
    return (coords)
coordinates = tiles.geometry.apply(coord_lister)

In [ ]:
for i,coor in enumerate(coordinates):
    area1 = ee.Geometry.Polygon(coor)
    filename_sentinel = os.path.join(out_dir, f'{date_sentinel}_sentinel_kizil_{i}.tif')
    sentinel_clipeed = sentinel.clip(area1).unmask()
    geemap.ee_export_image(sentinel_clipeed, filename=filename_sentinel, scale=10, file_per_band=False)

In [ ]:
list(coordinates[0])

### Add Ramsar to Map

In [ ]:
ramsar_shp = '/home/cak/Desktop/lake_extraction/Data/sulakalanlar/ramsar_ist.shp'
ramsar = geemap.shp_to_ee(ramsar_shp)
Map.addLayer(ramsar, {}, 'Ramsar')

## Draw Endmembers

In [ ]:
# Map.draw_features

In [ ]:
# water_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# forest_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# soil_ee = ee.FeatureCollection(Map.draw_features)

In [ ]:
# geemap.ee_to_shp(water_ee, filename='./exported/water.shp')

In [ ]:
# geemap.ee_to_shp(forest_ee, filename='./exported/forest.shp')

In [ ]:
# geemap.ee_to_shp(soil_ee, filename='./exported/soil.shp')

### Read Endmembers

In [ ]:
water_shp = '../Data/ML/SWM_water_selected_4326.shp'
water = geemap.shp_to_ee(water_shp)
Map.addLayer(water, {}, 'Water')

forest_shp = '../Data/Endmembers/forest_kizil.shp'
forest = geemap.shp_to_ee(forest_shp)
Map.addLayer(forest, {}, 'forest')

soil_shp = '../Data/Endmembers/soil_kizil.shp'
soil = geemap.shp_to_ee(soil_shp)
Map.addLayer(soil, {}, 'soil')

### Endmember Zonal Statistics

In [ ]:
waterMean = sentinel.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': water.geometry().getInfo(),
        'scale': 20,
        'maxPixels': 1e9
    }).values()

forestMean = sentinel.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': forest.geometry().getInfo(),
        'scale': 20,
        'maxPixels': 1e9
    }).values()

soilMean = sentinel.reduceRegion(
    **{
        'reducer': ee.Reducer.mean(),
        'geometry': soil.geometry().getInfo(),
        'scale': 20,
        'maxPixels': 1e9
    }).values()

In [ ]:
waterMean.getInfo()

In [ ]:
endmembers = ee.Array.cat([waterMean , forestMean , soilMean] , 1)

In [ ]:
arrayImage = sentinel.toArray().toArray(1)

unmixed = ee.Image(endmembers).matrixSolve(arrayImage)

unmixedImage = unmixed.arrayProject([0]).arrayFlatten([['water','forest','soil']])

In [ ]:
Map.addLayer(unmixedImage, {}, 'Unmixed')

In [ ]:
waterBand = unmixedImage.select('water').gt(0.95)

Map.addLayer(waterBand, {} , 'waterBand')

In [ ]:
classes = waterBand.reduceToVectors(
**{
  'reducer': ee.Reducer.countEvery(), 
  'geometry': waterBand.geometry().getInfo(), 
  'scale': 30,
  'maxPixels': 1e8
});


In [ ]:
result = ee.FeatureCollection(classes);
Map.addLayer(result,{},'Water_Vector');

In [ ]:
fractions = sentinel.unmix([waterMean, forestMean, soilMean])

In [ ]:
Map.addLayer(fractions, {}, 'unmixed');


### Export Classification

In [ ]:
out_dir = '../Data/Unmixed'

for i,coor in enumerate(coordinates):
    area1 = ee.Geometry.Polygon(coor)
    unmixedImage1 = fractions.clip(area1).unmask()
    filename = os.path.join(out_dir, f'Unmixed_{i}.tif')
    geemap.ee_export_image(unmixedImage1, filename=filename, scale=10, file_per_band=False)

# Supervised classification


In [ ]:
# clip image
# image_roi = image.clip(roi.geometry())
# Map.addLayer(image_roi,landsat_vis,'Cropped_Landsat')

In [ ]:
bands = ['B2','B3','B4','B5','B6','B7','B8','B11','B12','NDVI','NDWI']
# bands.extend(['NDVI','NDWI'])
print(bands)

In [ ]:
sentinel.getInfo()

In [ ]:
waterf = ee.Feature(water.geometry(), {'class': 0, 'name': 'water'});
forestf = ee.Feature(forest.geometry(), {'class': 1, 'name': 'forest'});
soilf = ee.Feature(soil.geometry(), {'class': 2, 'name': 'soil'});

In [ ]:
trainingFeatures = ee.FeatureCollection([waterf, forestf, soilf])

In [ ]:
classifierTraining = sentinel.select(bands).sampleRegions(
      collection= trainingFeatures, 
      properties= ['class'], 
      scale= 10
    );

In [ ]:
# // Randomly split the data into 60% for training, and 40% for testing
trainingTesting = classifierTraining.randomColumn('random',111009);

training = trainingTesting.filter(ee.Filter.lt('random', 0.6));

testing = trainingTesting.filter(ee.Filter.gte('random', 0.6));

### Non-linear regression functions

In [ ]:
cartclassifier = ee.Classifier.cart(randomSeed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    );

In [ ]:
cartClasifficationImage = sentinel.select(bands).classify(cartclassifier);

Map.addLayer(cartClasifficationImage, {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']},'CART classification');

In [ ]:
rfClassification = ee.Classifier.smileRandomForest(numberOfTrees=15, seed=111009).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

In [ ]:
# // Perform the RF regression on the landsat image
rfClassificationImage = sentinel.select(bands).classify(rfClassification);
    
# // Visualize the RF regression
# Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
#                                    'palette':['blue','green', 'yellow']}, 'RF classification');

Map.addLayer(rfClassificationImage,  {'min': 0, 'max': 2,
                                   'palette':['blue','green', 'yellow']}, 'RF classification');

In [ ]:
# ee.Classifier.libsvm(decisionProcedure, svmType, kernelType, shrinking, degree, gamma, coef0, cost, nu, terminationEpsilon, lossEpsilon, oneClass)

# // Create an SVM classifier with custom parameters.
svClassification = ee.Classifier.libsvm(kernelType='RBF',gamma=1,cost=100).train(
      features= training, 
      classProperty= 'class', 
      inputProperties= bands
    )

In [ ]:
# // Perform the RF regression on the landsat image
svClassificationImage = sentinel.select(bands).classify(svClassification);
    
# // Visualize the RF regression
Map.addLayer(svClassificationImage,{'min': 0, 'max': 2,
                                   'palette':['blue', 'green','yellow']}, 'SV CLassification');

In [ ]:
out_dir = '../Data/ML'
filename_rf = os.path.join(out_dir, 'RF.tif')
rfClassificationImage = rfClassificationImage.clip(area).unmask()
geemap.ee_export_image(rfClassificationImage, filename=filename_rf, scale=10, file_per_band=False)

In [ ]:
for i,coor in enumerate(coordinates):
    area1 = ee.Geometry.Polygon(coor)
    cartClasifficationImage1 = cartClasifficationImage.clip(area1).unmask()
    svClassificationImage1 = svClassificationImage.clip(area1).unmask()
    
    filename_cart = os.path.join(out_dir, f'Cart_{i}.tif')
    filename_svm = os.path.join(out_dir, f'SWM_{i}.tif')
    
    geemap.ee_export_image(cartClasifficationImage1, filename=filename_cart, scale=10, file_per_band=False)
    geemap.ee_export_image(svClassificationImage1, filename=filename_svm, scale=10, file_per_band=False)

In [ ]:
folder = '/home/cak/Desktop/lake_extraction/Data/Unmixed'
os.chdir(folder)
if os.path.exists('merged.tif'):os.remove('merged.tif')
files = glob.glob1(folder,'*.tif')
files = ' '.join(files)
code = f"gdal_merge.py -o merged.tif {files}"
os.system(code)

In [ ]:
A = 2
B = 1
C = -0.8

if os.path.exists('merged_classified.tif'):os.remove('merged_classified.tif')

code =  f"""gdal_calc.py --calc "logical_and(A<{A},\
B<{B},C>{C})" --format GTiff --type Float32 -A \
/home/cak/Desktop/lake_extraction/Data/Unmixed/merged.tif \
--A_band 1 -B /home/cak/Desktop/lake_extraction/Data/Unmixed/merged.tif \
--B_band 2 -C /home/cak/Desktop/lake_extraction/Data/Unmixed/merged.tif \
--C_band 3 --outfile /home/cak/Desktop/lake_extraction/Data/Unmixed/merged_classified.tif
"""
os.system(code)

if os.path.exists('merged_classified.shp'):os.remove('merged_classified.shp')
wbt = '/home/cak/Desktop/lake_extraction/Geemap/WBT/whitebox_tools'
code = f"""{wbt} -r=RasterToVectorPolygons -v --wd="." --input=merged_classified.tif -o=../../Data/Unmixed/merged_classified.shp"""
os.system(code)



In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd